# Cuaderno para simular escenarios de recrédito inhabilitado

Este cuaderno tiene las simulaciones de escenarios para recrédito inhabilitado tomando los factores de cálculo para monto bruto dados por un cociente entre la distribución uniforme de tasa de ganancia cancelada sobre nueva.

## Leer cartera julio y simular

In [1]:
import pandas as pd
import warnings
import math
warnings.filterwarnings("ignore")
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys
sys.path.append(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\codigos")

from utils import ingresos_financieros, ingreso_fin_entre_fechas, meses_entre_fechas

In [2]:
mes_simulacion = 10
nombres_mes_simulacion = "OCTUBRE"
#fac_ago = pd.read_excel(r"C:\Users\mariajose_chinchilla\Desktop\Agosto\Factores recrédito inhabilitado.xlsx", sheet_name="Agosto")
#fac_sep = pd.read_excel(r"C:\Users\mariajose_chinchilla\Desktop\Agosto\Factores recrédito inhabilitado.xlsx", sheet_name="Septiembre")
#fac_ago.fillna(0, inplace=True)
#fac_sep.fillna(0, inplace=True)

In [3]:
# cartera
cartera = pd.read_excel("../db/cartera julio v3.xlsx")
cartera = cartera[cartera["TIPO_PAGO"] == "REMESA"]
cartera = cartera[cartera["SALDO"] > 0]

In [4]:
cartera["FECHA_APERTURA"] = pd.to_datetime(cartera["FECHA_APERTURA"])
cartera["MESES TRANSCURRIDOS"] = cartera.apply(lambda x: 12 * relativedelta(datetime.today(), x["FECHA_APERTURA"]).years + relativedelta(datetime.today(), x["FECHA_APERTURA"]).months, axis=1)
cartera["MESES TRANSCURRIDOS"] = cartera["MESES TRANSCURRIDOS"].astype("int64")

In [5]:
def porcentaje_comision_desembolso(x):
    if x < 125000:
        porcentaje = 0.05
    elif x >= 125000:
        porcentaje = 0.02
    return porcentaje

cartera["PORCENTAJE COMISION DESEMBOLSO"] = cartera.apply(lambda x: porcentaje_comision_desembolso(x["DESEMBOLSO_NETO"]), axis=1)
cartera["COMISION PENDIENTE"] = cartera.apply(lambda x: ingreso_fin_entre_fechas(x["DESEMBOLSO_NETO"], x["PORCENTAJE COMISION DESEMBOLSO"]/12, x["PLAZO"], x["MESES TRANSCURRIDOS"],  x["MESES TRANSCURRIDOS"] + 12 - mes_simulacion), axis=1)

In [6]:
def paga_reservas(x):
    if x < 0.15:
        return 1
    elif x >= 0.15:
        return 0
    
def tpp(factor_tasa, montos):
    return sum(factor_tasa) / sum(montos)
    
cartera["PORCENTAJE SALDO"] = 1- cartera["SALDO"] / cartera["MONTO DESEMBOLSADO"]
cartera["FACTOR TASA"] = cartera["TASA"] * cartera["MONTO DESEMBOLSADO"]
cartera_agrupado = cartera.groupby(by="CODIGO_CLIENTE").agg(PORCENTAJE_MINIMO=("PORCENTAJE SALDO", "min"),
                                                            DEUDA_INTERNA=("SALDO", "sum"),
                                                            CREDITOS_A_CANCELAR=("NO_CREDITO", list)).reset_index()
cartera_agrupado.columns = ["CODIGO_CLIENTE", "PORCENTAJE MINIMO", "DEUDA_INTERNA", "CREDITOS A CANCELAR"]
cartera_agrupado["PAGA RESERVAS"] = cartera_agrupado.apply(lambda x: paga_reservas(x["PORCENTAJE MINIMO"]), axis=1)
cartera = pd.merge(cartera, cartera_agrupado, how="left", on="CODIGO_CLIENTE")

cartera_temp = cartera.groupby(by="CODIGO_CLIENTE").apply(lambda x: tpp(x["FACTOR TASA"], x["MONTO DESEMBOLSADO"])).reset_index()
cartera_temp.columns = ["CODIGO_CLIENTE", "TPP CLIENTE"]
cartera = pd.merge(cartera, cartera_temp, how="left", on="CODIGO_CLIENTE")

In [7]:
cartera["INGRESOS A CANCELAR"] = cartera.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12 - mes_simulacion), axis=1)
alpha = (0.0747+0.0025) / (1 - 0.146)
cartera["PASIVOS A CANCELAR"] = cartera.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO DESEMBOLSADO"], alpha/12, x["PLAZO"], x["MESES TRANSCURRIDOS"] + 12 - mes_simulacion, x["MESES TRANSCURRIDOS"]), axis=1)
cartera_temp = cartera.groupby(by="CODIGO_CLIENTE").agg(PASIVOS_TOTALES_A_CANCELAR_CLIENTE=("PASIVOS A CANCELAR", "sum"),
                                                        INGRESOS_TOTALES_A_CANCELAR=("INGRESOS A CANCELAR", "sum"),
                                                        COMISIONES_A_TRAERSE=("COMISION PENDIENTE", "sum")).reset_index()
cartera_temp.columns = ["CODIGO_CLIENTE", "PASIVOS TOTALES A CANCELAR CLIENTE", "INGRESOS TOTALES A CANCELAR", "COMISIONES A TRAERSE"]
cartera = pd.merge(cartera, cartera_temp, how="left", on="CODIGO_CLIENTE")

In [ ]:
# # calcular el monto que se le ofertaria
# def calcular_monto_oferta(tasa_actual, df_factores):
#     cercana = math.floor(tasa_actual)
#     try:
#         if cercana < 18:
#             return 0
#         if cercana >= 18 and cercana <= 24:
#             return df_factores.loc[df_factores["Tasa"] == cercana, cercana].values[0]
#         if cercana > 24:
#             return df_factores.loc[df_factores["Tasa"] == 24, 24].values[0]
#     except Exception as e:
#         print(e)
#         print(cercana)


# cartera["FACTOR BRUTO OFERTA"] = cartera.apply(lambda x: calcular_monto_oferta(x["TPP CLIENTE"], fac_ago), axis=1)
# cartera["MONTO BRUTO OFERTA"] = cartera["FACTOR BRUTO OFERTA"] * cartera["SALDO"]
# cartera["MONTO BRUTO OFERTA"] = cartera.apply(lambda x: min(x["MONTO BRUTO OFERTA"], 300000), axis=1)
# cartera["RESERVAS POR REESTRUCTURA"] = cartera["PAGA RESERVAS"] * 0.05 * 0.45 * cartera["MONTO BRUTO OFERTA"]

In [12]:
cartera[cartera["CODIGO_CLIENTE"] == 101056391]["DESEMBOLSO_NETO"]

37477     108000.00
138677    124627.77
178097     13900.00
195412    207200.00
Name: DESEMBOLSO_NETO, dtype: float64

In [ ]:
resultados = cartera.copy()
resultados = resultados.drop_duplicates(subset="CODIGO_CLIENTE")
resultados[f"INGRESOS FINANCIERSO A {nombres_mes_simulacion}"] = resultados.apply(lambda x: ingresos_financieros(x["MONTO BRUTO OFERTA"], x["TPP CLIENTE"]/1200, x["PLAZO"], 12 - mes_simulacion), axis=1)

In [ ]:
resultados["NUEVOS PASIVOS EN EL AÑO"] = resultados.apply(lambda x: ingresos_financieros(x["MONTO BRUTO OFERTA"], alpha/12, x["PLAZO"], 12 - mes_simulacion), axis=1)
resultados["PASIVOS NUEVOS TOTALES"] = resultados.apply(lambda x: ingresos_financieros(x["MONTO BRUTO OFERTA"], alpha/12, x["PLAZO"], x["PLAZO"]), axis=1)

In [ ]:
resultados = resultados[(resultados["PAGA RESERVAS"] == 1) & (resultados["MONTO BRUTO OFERTA"] < 300000)
                    & (resultados["MONTO BRUTO OFERTA"] > 0)]

In [ ]:
ingresos_en_el_año = resultados[f"INGRESOS FINANCIERSO A {nombres_mes_simulacion}"].sum()
ingresos_cancelados = resultados["INGRESOS TOTALES A CANCELAR"].sum()
pasivos_nuevos = resultados["NUEVOS PASIVOS EN EL AÑO"].sum()
pasivos_a_cancelar = resultados["PASIVOS A CANCELAR"].sum()
reservas_reestructura = resultados["RESERVAS POR REESTRUCTURA"].sum()
comisiones_a_traerse = resultados["COMISIONES A TRAERSE"].sum()

In [ ]:
resultados["MARGEN CANCELADO"] = resultados["INGRESOS TOTALES A CANCELAR"] - resultados["PASIVOS A CANCELAR"]
resultados["MARGEN NUEVO"] = resultados[f"INGRESOS FINANCIERSO A {nombres_mes_simulacion}"] - resultados["RESERVAS POR REESTRUCTURA"] - resultados["NUEVOS PASIVOS EN EL AÑO"]
resultados["DIFERENCIA MARGEN"] = resultados["MARGEN NUEVO"] - resultados["MARGEN CANCELADO"] 

In [ ]:
a = resultados[(resultados["DIFERENCIA MARGEN"] > 0) & (resultados["RCI_INTERNO"].isna() | resultados["RCI_INTERNO"] < 50)]

In [ ]:
vars = [f"INGRESOS FINANCIERSO A {nombres_mes_simulacion}",
"INGRESOS TOTALES A CANCELAR",
"NUEVOS PASIVOS EN EL AÑO",
"MONTO BRUTO OFERTA",
"PASIVOS A CANCELAR",
"RESERVAS POR REESTRUCTURA",
"COMISIONES A TRAERSE",
"MARGEN CANCELADO",
"MARGEN NUEVO",
"DIFERENCIA MARGEN"]
for var in vars:
    print(f"{var} es {int(a[var].sum() / 10**6)}")

In [ ]:
final = a[["CODIGO_CLIENTE", "DPI", "RCI_INTERNO", "SCORE", "SCORE_RIESGO",
       "DEUDA_INTERNA",
       "TPP CLIENTE", "INGRESOS TOTALES A CANCELAR", 
       "PASIVOS TOTALES A CANCELAR CLIENTE", "COMISIONES A TRAERSE",
       "FACTOR BRUTO OFERTA", "MONTO BRUTO OFERTA",
       "RESERVAS POR REESTRUCTURA", "INGRESOS FINANCIERSO A AGOSTO",
       "NUEVOS PASIVOS EN EL AÑO", "MARGEN CANCELADO", "MARGEN NUEVO",
       "DIFERENCIA MARGEN", "PLAZO", "CREDITOS A CANCELAR"]]

final.rename(columns={"INGRESOS TOTALES A CANCELAR": "INGRESOS DEL AÑO A CANCELAR POR RECREDITO",
                  "PASIVOS TOTALES A CANCELAR CLIENTE": "PASIVOS QUE YA NO SE PAGARIAN EN EL AÑO", 
                  "FACTOR BRUTO OFERTA": "FACTOR INICIATIVA", "MONTO BRUTO OFERTA": "MONTO BRUTO A OFERTAR",
                  "RESERVAS POR REESTRUCTURA": "PE(AD) RESERVAS DE A HACIA B",
                  "INGRESOS FINANCIERSO A AGOSTO": "INGRESOS NUEVOS EN EL AÑO POR RECREDITO",
                  "NUEVOS PASIVOS EN EL AÑO": "PASIVOS DEL RECREDITO EN EL AÑO"}, inplace=True)

In [ ]:
# compra_saldo = pd.read_excel("../db/compraSaldo.xlsx")
# liquidez = pd.read_excel("../db/liquidez.xlsx")
# todos = pd.concat([compra_saldo, liquidez])

In [ ]:
# final = pd.merge(a, todos, how="inner", left_on="DPI", right_on="DxDpi")

In [ ]:
# final = final[final["MONTO BRUTO OFERTA"] <= final["DfMontoMaximoOfertado"]]

In [ ]:
# final = final[["CODIGO_CLIENTE", "DPI", "DEUDA_INTERNA", "TPP CLIENTE", 
#                "SCORE", "PLAZO", "MONTO BRUTO OFERTA"]]

# final.to_excel("../db/Ofertas_recredito_inhabiltitado.xlsx")

In [ ]:
# b = a[["CODIGO_CLIENTE", "DPI", "RCI_INTERNO", "SCORE", "SCORE_RIESGO",
#        "DEUDA_INTERNA",
#        "TPP CLIENTE", "INGRESOS TOTALES A CANCELAR", 
#        "PASIVOS TOTALES A CANCELAR CLIENTE", "COMISIONES A TRAERSE",
#        "FACTOR BRUTO OFERTA", "MONTO BRUTO OFERTA",
#        "RESERVAS POR REESTRUCTURA", "INGRESOS FINANCIERSO A AGOSTO",
#        "NUEVOS PASIVOS EN EL AÑO", "MARGEN CANCELADO", "MARGEN NUEVO",
#        "DIFERENCIA MARGEN", "PLAZO"]]

In [ ]:
# b.rename(columns={"INGRESOS TOTALES A CANCELAR": "INGRESOS DEL AÑO A CANCELAR POR RECREDITO",
#                   "PASIVOS TOTALES A CANCELAR CLIENTE": "PASIVOS QUE YA NO SE PAGARIAN EN EL AÑO", 
#                   "FACTOR BRUTO OFERTA": "FACTOR INICIATIVA", "MONTO BRUTO OFERTA": "MONTO BRUTO A OFERTAR",
#                   "RESERVAS POR REESTRUCTURA": "PE(AD) RESERVAS DE A HACIA B",
#                   "INGRESOS FINANCIERSO A AGOSTO": "INGRESOS NUEVOS EN EL AÑO POR RECREDITO",
#                   "NUEVOS PASIVOS EN EL AÑO": "PASIVOS DEL RECREDITO EN EL AÑO"}, inplace=True)

In [ ]:
# b.to_excel("../db/Clientes iniciativa habilitación recrédito.xlsx")

In [ ]:
cartera["FACTOR TASA"].sum() / cartera["MONTO DESEMBOLSADO"].sum()